In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import cv2
import random
from sklearn.model_selection import train_test_split

# Crear el modelo CNN con base de transferencias y mejora de rendimiento
def crear_modelo_cnn():
    modelo = Sequential()

    # Capa convolucional
    modelo.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    modelo.add(MaxPooling2D(pool_size=(2, 2)))

    # Más capas convolucionales
    modelo.add(Conv2D(64, (3, 3), activation='relu'))
    modelo.add(MaxPooling2D(pool_size=(2, 2)))

    # Aplanar la salida antes de la capa densa
    modelo.add(Flatten())

    # Capas densas
    modelo.add(Dense(128, activation='relu'))
    modelo.add(Dropout(0.5))  # Dropout para evitar el sobreajuste
    modelo.add(Dense(3, activation='softmax'))  # 3 clases: rojo, amarillo, verde

    modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return modelo

# Función para crear una imagen de semáforo simulada
def generar_imagen_semaforo(color, tamano=(224, 224)):
    # Crear una imagen negra de fondo
    imagen = np.zeros((tamano[0], tamano[1], 3), dtype=np.uint8)

    # Definir el centro y el radio del círculo
    centro = (tamano[1] // 2, tamano[0] // 2)
    radio = tamano[0] // 5

    # Colores posibles en BGR (azul, verde, rojo)
    if color == "rojo":
        cv2.circle(imagen, centro, radio, (0, 0, 255), -1)  # Rojo
    elif color == "amarillo":
        cv2.circle(imagen, centro, radio, (0, 255, 255), -1)  # Amarillo
    elif color == "verde":
        cv2.circle(imagen, centro, radio, (0, 255, 0), -1)  # Verde

    # Añadir algo de ruido para hacerlo más realista (opcional)
    for _ in range(random.randint(50, 200)):  # Puntos de ruido aleatorios
        x, y = random.randint(0, tamano[1] - 1), random.randint(0, tamano[0] - 1)
        if random.random() > 0.9:  # Solo un pequeño porcentaje de ruido
            imagen[y, x] = [random.randint(0, 255) for _ in range(3)]  # Rellenar con colores aleatorios

    # Simular alguna variabilidad (translación, escala o cambio de brillo)
    if random.random() > 0.5:  # Aleatorio para mover la imagen
        M = np.float32([[1, 0, random.randint(-10, 10)], [0, 1, random.randint(-10, 10)]])  # Traslación
        imagen = cv2.warpAffine(imagen, M, tamano)

    if random.random() > 0.7:  # Aleatorio para cambiar el brillo
        factor = random.uniform(0.5, 1.5)  # Brillo variable
        imagen = np.clip(imagen * factor, 0, 255).astype(np.uint8)

    return imagen

# Generar los datos de entrenamiento
def preparar_datos_entrenamiento():
    X_train = []  # Almacenar imágenes de entrenamiento preprocesadas
    y_train = []  # Almacenar las etiquetas correspondientes (0 = rojo, 1 = amarillo, 2 = verde)

    # Crear 1 imágenes por cada color de semáforo (rojo, amarillo, verde)
    for i in range(3):
        imagen_rojo = generar_imagen_semaforo("rojo")
        imagen_amarillo = generar_imagen_semaforo("amarillo")
        imagen_verde = generar_imagen_semaforo("verde")

        X_train.append(imagen_rojo)
        y_train.append(0)  # Rojo

        X_train.append(imagen_amarillo)
        y_train.append(1)  # Amarillo

        X_train.append(imagen_verde)
        y_train.append(2)  # Verde

    # Convertir a arrays de numpy
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Normalizar las imágenes
    X_train = X_train / 255.0  # Normalizamos a rango [0, 1]

    # Convertir etiquetas a codificación one-hot
    y_train_onehot = tf.keras.utils.to_categorical(y_train, 3)

    return X_train, y_train_onehot

# Crear generador de datos con augmentación
def crear_generador():
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    return datagen

# Entrenar el modelo con EarlyStopping
def entrenar_red():
    X_train, y_train = preparar_datos_entrenamiento()

    # Dividir los datos en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Crear generador de datos para entrenamiento
    datagen_train = crear_generador()
    datagen_train.fit(X_train)

    # Crear generador de datos para validación
    datagen_val = ImageDataGenerator(rescale=1./255)
    datagen_val.fit(X_val)

    modelo = crear_modelo_cnn()

    # Early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Entrenar el modelo
    modelo.fit(datagen_train.flow(X_train, y_train, batch_size=10),
               epochs=50,
               validation_data=datagen_val.flow(X_val, y_val, batch_size=10),
               callbacks=[early_stop],
               verbose=1)

    # Guardar el modelo entrenado
    modelo.save("modelo_semaforo_cnn.h5")
    print("Modelo guardado como 'modelo_semaforo_cnn.h5'")

if __name__ == "__main__":
    entrenar_red()


C:\Users\Equipo\PycharmProjects\JupyterProject\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


C:\Users\Equipo\PycharmProjects\JupyterProject\.venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.2857 - loss: 1.1094 - val_accuracy: 0.0000e+00 - val_loss: 1.1191
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step - accuracy: 0.5714 - loss: 0.6191 - val_accuracy: 0.0000e+00 - val_loss: 1.1184
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.8571 - loss: 0.3505 - val_accuracy: 0.0000e+00 - val_loss: 1.1169
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.7143 - loss: 2.1723 - val_accuracy: 0.0000e+00 - val_loss: 1.1406
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step - accuracy: 0.8571 - loss: 0.1898 - val_accuracy: 0.0000e+00 - val_loss: 1.1231
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.7143 - loss: 0.8635 - val_accuracy: 0.0000e+00 - val_loss: 1.1316
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 415ms/step - accuracy: 1.0000 - loss: 0.0479 - val_accuracy: 0.0000e+00 - val_loss: 1.1270
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 413ms/step - accuracy: 0.7143 - loss: 0.6124 - val_accuracy: 0.00

Modelo guardado como 'modelo_semaforo_cnn.h5'


In [ ]:
import tensorflow as tf
import cv2
import numpy as np

# Cargar el modelo previamente entrenado
modelo = tf.keras.models.load_model("modelo_semaforo_cnn1.h5")

# Función para preprocesar la imagen y obtener el color promedio
def obtener_color_promedio(frame):
    # Reducir el ruido con un filtro GaussianBlur
    frame = cv2.GaussianBlur(frame, (5, 5), 0)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Rango para los colores (ajustar según la iluminación y el ruido)
    rango_rojo = ((0, 100, 100), (10, 255, 255))
    rango_amarillo = ((20, 100, 100), (40, 255, 255))
    rango_verde = ((40, 100, 100), (80, 255, 255))

    # Aplicar umbral adaptativo
    rojo = cv2.inRange(hsv, rango_rojo[0], rango_rojo[1])
    amarillo = cv2.inRange(hsv, rango_amarillo[0], rango_amarillo[1])
    verde = cv2.inRange(hsv, rango_verde[0], rango_verde[1])

    # Calcular el promedio de los colores
    promedio_rojo = cv2.countNonZero(rojo)
    promedio_amarillo = cv2.countNonZero(amarillo)
    promedio_verde = cv2.countNonZero(verde)

    # Normalizar los valores para asegurar que estén en el rango adecuado
    return np.array([promedio_rojo / (frame.size / 3), promedio_amarillo / (frame.size / 3), promedio_verde / (frame.size / 3)])

# Función para predecir el color del semáforo en tiempo real
def predecir_color(frame):
    # Preprocesar la imagen
    imagen_preprocesada = cv2.resize(frame, (224, 224))  # Cambiar el tamaño de la imagen
    imagen_preprocesada = imagen_preprocesada / 255.0  # Normalizar la imagen

    # Hacer la predicción
    prediccion = modelo.predict(np.array([imagen_preprocesada]), verbose=0)
    clase_predicha = np.argmax(prediccion)

    if clase_predicha == 0:
        return "Rojo: pare"
    elif clase_predicha == 1:
        return "Amarillo: este alerta"
    elif clase_predicha == 2:
        return "Verde: continue"
    return None

# Iniciar la cámara y usar el modelo para predecir el color del semáforo
def iniciarCamara():
    print("Iniciando módulo de cámara...")
    cam = cv2.VideoCapture(0)

    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return

    print("Cámara abierta")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al capturar imagen")
            break

        # Predicción en tiempo real
        color_predicho = predecir_color(frame)
        print(f"Color del semáforo: {color_predicho}")

        # Mostrar el resultado en la imagen
        cv2.putText(frame, f"Color: {color_predicho}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Camara", frame)

        # Romper el bucle si el usuario presiona 'Esc'
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC
            break

    cam.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    iniciarCamara()


Iniciando módulo de cámara...
Cámara abierta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: este alerta
Color del semáforo: Amarillo: e

In [1]:
import numpy as np
import cv2

# Función de activación para la red Hopfield
def activacion(patron):
    return np.where(patron >= 0, 1, -1)

# Red Hopfield
class RedHopfield:
    def __init__(self, tamaño):
        self.tamaño = tamaño
        self.pesos = np.zeros((tamaño, tamaño))

    def entrenar(self, patrones):
        print("Entrenando la red Hopfield...")
        for idx, patron in enumerate(patrones):
            patron = patron.reshape(-1, 1)
            self.pesos += np.dot(patron, patron.T)
            np.fill_diagonal(self.pesos, 0)  # Las neuronas no se conectan entre sí
            print(f"\nPesos después de entrenar el patrón {idx + 1} ({['Rojo', 'Amarillo', 'Verde'][idx]}):")
            print(self.pesos)

    def reconocer(self, entrada):
        entrada = entrada.reshape(-1, 1)
        salida = np.copy(entrada)
        iteracion = 0
        while True:
            iteracion += 1
            salida_anterior = salida.copy()
            salida = activacion(np.dot(self.pesos, salida))

            # Imprimir los pesos y la salida en cada iteración
            print(f"\nIteración {iteracion} de reconocimiento:")
            print("Pesos actuales:")
            print(self.pesos)
            print("Salida actual:")
            print(salida.flatten())

            if np.array_equal(salida, salida_anterior):
                break
        return salida.flatten()

# Crear los patrones para los colores del semáforo
rojo = np.array([1, -1, -1, -1, 1, -1, -1, -1, 1])  # Color Rojo
amarillo = np.array([-1, 1, -1, 1, -1, 1, -1, 1, -1])  # Color Amarillo
verde = np.array([-1, -1, 1, -1, -1, 1, 1, -1, -1])  # Color Verde

# Entrenamiento
patrones = [rojo, amarillo, verde]
red = RedHopfield(tamaño=9)
red.entrenar(patrones)

# Guardar los pesos entrenados
np.save('pesos_hopfield.npy', red.pesos)
print("\nEntrenamiento completado. Pesos guardados como 'pesos_hopfield.npy'.\n")

# Ejemplo de reconocimiento
entrada = np.array([-1, -1, -1, -1, 1, 1, -1, -1, -1])  # Ejemplo de entrada para "rojo"
print("Reconociendo la entrada:")
reconocimiento = red.reconocer(entrada)
print(f"\nPatrón reconocido: {reconocimiento}")


Entrenando la red Hopfield...

Pesos después de entrenar el patrón 1 (Rojo):
[[ 0. -1. -1. -1.  1. -1. -1. -1.  1.]
 [-1.  0.  1.  1. -1.  1.  1.  1. -1.]
 [-1.  1.  0.  1. -1.  1.  1.  1. -1.]
 [-1.  1.  1.  0. -1.  1.  1.  1. -1.]
 [ 1. -1. -1. -1.  0. -1. -1. -1.  1.]
 [-1.  1.  1.  1. -1.  0.  1.  1. -1.]
 [-1.  1.  1.  1. -1.  1.  0.  1. -1.]
 [-1.  1.  1.  1. -1.  1.  1.  0. -1.]
 [ 1. -1. -1. -1.  1. -1. -1. -1.  0.]]

Pesos después de entrenar el patrón 2 (Amarillo):
[[ 0. -2.  0. -2.  2. -2.  0. -2.  2.]
 [-2.  0.  0.  2. -2.  2.  0.  2. -2.]
 [ 0.  0.  0.  0.  0.  0.  2.  0.  0.]
 [-2.  2.  0.  0. -2.  2.  0.  2. -2.]
 [ 2. -2.  0. -2.  0. -2.  0. -2.  2.]
 [-2.  2.  0.  2. -2.  0.  0.  2. -2.]
 [ 0.  0.  2.  0.  0.  0.  0.  0.  0.]
 [-2.  2.  0.  2. -2.  2.  0.  0. -2.]
 [ 2. -2.  0. -2.  2. -2.  0. -2.  0.]]

Pesos después de entrenar el patrón 3 (Verde):
[[ 0. -1. -1. -1.  3. -3. -1. -1.  3.]
 [-1.  0. -1.  3. -1.  1. -1.  3. -1.]
 [-1. -1.  0. -1. -1.  1.  3. -1. -1.]
 [-

In [ ]:
import cv2
import numpy as np

# Función de activación para la red Hopfield
def activacion(patron):
    return np.where(patron >= 0, 1, -1)

# Red Hopfield
class RedHopfield:
    def __init__(self, tamaño, pesos):
        self.tamaño = tamaño
        self.pesos = pesos

    def reconocer(self, entrada):
        entrada = entrada.reshape(-1, 1)
        salida = np.copy(entrada)
        while True:
            salida_anterior = salida.copy()
            salida = activacion(np.dot(self.pesos, salida))
            if np.array_equal(salida, salida_anterior):
                break
        return salida.flatten()

# Función para identificar el color basado en la entrada
def identificar_color(frame):
    # Convertir la imagen a espacio de color HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Definir los rangos de color para rojo, amarillo y verde
    # Rojo
    rojo_bajo = np.array([0, 120, 70])
    rojo_alto = np.array([10, 255, 255])
    rojo_mask = cv2.inRange(hsv, rojo_bajo, rojo_alto)

    # Amarillo
    amarillo_bajo = np.array([20, 100, 100])
    amarillo_alto = np.array([40, 255, 255])
    amarillo_mask = cv2.inRange(hsv, amarillo_bajo, amarillo_alto)

    # Verde
    verde_bajo = np.array([35, 50, 50])
    verde_alto = np.array([85, 255, 255])
    verde_mask = cv2.inRange(hsv, verde_bajo, verde_alto)

    # Contar los píxeles de cada color
    rojo_cont = np.sum(rojo_mask)
    amarillo_cont = np.sum(amarillo_mask)
    verde_cont = np.sum(verde_mask)

    # Decidir el color predominante
    if rojo_cont > max(amarillo_cont, verde_cont):
        return np.array([1, -1, -1, -1, 1, -1, -1, -1, 1])  # Rojo
    elif verde_cont > max(rojo_cont, amarillo_cont):
        return np.array([-1, -1, 1, -1, -1, 1, 1, -1, -1])  # Verde
    else:
        return np.array([-1, 1, -1, 1, -1, 1, -1, 1, -1])  # Amarillo

def iniciarCamara():
    print("Iniciando módulo")
    cam = cv2.VideoCapture(0)
    if not cam.isOpened():
        print("No se pudo abrir la cámara")
        return
    print("Cámara abierta")

    # Cargar los pesos entrenados
    try:
        pesos_hopfield = np.load('pesos_hopfield.npy')
        print("Pesos de la red Hopfield cargados correctamente.")
    except Exception as e:
        print(f"Error al cargar los pesos entrenados: {e}")
        return

    # Crear la red Hopfield con los pesos cargados
    red = RedHopfield(tamaño=9, pesos=pesos_hopfield)

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error al encontrar imagen")
            break

        # Extraer color y predecir usando la red Hopfield
        entrada = identificar_color(frame)
        color_identificado = red.reconocer(entrada)

        # Determinar el color predicho
        if np.array_equal(color_identificado, np.array([1, -1, -1, -1, 1, -1, -1, -1, 1])):  # Rojo
            texto = "Semáforo: Rojo - Deténgase"
        elif np.array_equal(color_identificado, np.array([-1, -1, 1, -1, -1, 1, 1, -1, -1])):  # Verde
            texto = "Semáforo: Verde - Puede avanzar"
        else:  # Amarillo
            texto = "Semáforo: Amarillo - Precaución"

        # Mostrar el texto en la imagen de la cámara
        cv2.putText(frame, texto, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Mostrar el frame con el texto
        cv2.imshow("Cámara", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # Escape para salir
            break

    cam.release()
    cv2.destroyAllWindows()

# Ejecutar la cámara
if __name__ == "__main__":
    iniciarCamara()


Iniciando módulo
Cámara abierta
Pesos de la red Hopfield cargados correctamente.
